# Obtaining robust cell population markers, and redefining/reassuring the biased cell populations

## imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import scanpy as sc
import scanpy.external as sce
import pandas as pd
import numpy as np
import os
import triku as tk
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm
import scipy.sparse as spr

In [ ]:
!pip install cellassign

In [ ]:
# local imports and imports from other notebooks
from cellassign import assign_cats
from fb_functions import make_gene_scoring_with_expr
%store -r dict_colors
%store -r seed
%store -r magma
%store -r data_dir

In [ ]:
mpl.rcParams['figure.dpi'] = 150
pd.options.display.float_format = "{:,.2f}".format

**IMPORTANT: I am running this analysis in a computer with ~500 GB of RAM. I will load many datasets at once, which might be too much for some computers. I took this decision conciously, to have as much info available at any time as possible. If you cannot run all the analysis at once, you can run it by parts.**

## Anndata loading

In [ ]:
deng_2021_dir = data_dir + '/deng_2021'
deng_2021_scar_fb = sc.read(deng_2021_dir + '/deng_2021_scar_fb_processed.h5')

In [ ]:
gao_2021_dir = data_dir + '/gao_2021'
gao_2021_ctrl_fb = sc.read(gao_2021_dir + '/gao_2021_ctrl_fb_processed.h5')

In [ ]:
gaydosik_2020_dir = data_dir + '/gaydosik_2020'
gaydosik_2020_ctrl_fb = sc.read(gaydosik_2020_dir + '/gaydosik_2020_ctrl_fb_processed.h5')

In [ ]:
he_2020_dir = data_dir + '/He_2020'
he_2020_ctrl_fb = sc.read(he_2020_dir + '/he_2020_ctrl_fb_processed.h5')

In [ ]:
hughes_2020_dir = data_dir + '/hughes_2020'
hughes_2020_ctrl_fb = sc.read(hughes_2020_dir + '/hughes_2020_ctrl_fb_processed.h5')

In [ ]:
kim_2020_dir = data_dir + '/Kim_2020'
kim_2020_ctrl_fb = sc.read(kim_2020_dir + '/kim_2020_ctrl_fb_processed.h5')

In [ ]:
kim_2021_dir = data_dir + '/kim_2021'
kim_2021_ctrl_fb = sc.read(kim_2021_dir + '/kim_2021_ctrl_fb_processed.h5')

In [ ]:
liu_2021_dir = data_dir + '/liu_2021'
liu_2021_ctrl_fb = sc.read(liu_2021_dir + '/liu_2021_ctrl_fb_processed.h5')

In [ ]:
mirizio_2020_dir = data_dir + '/mirizio_2020'
mirizio_2020_scl_fb = sc.read(mirizio_2020_dir + '/mirizio_2020_scl_2020_fb_processed.h5')

In [ ]:
reynolds_2021_dir = data_dir + '/reynolds_2021'
reynolds_2021_ctrl_fb = sc.read(reynolds_2021_dir + '/reynolds_2021_ctrl_fb_processed.h5')

In [ ]:
sole_2020_dir = data_dir + '/Sole-Boldo_2020'
sole_2020_young_fb = sc.read(sole_2020_dir + '/sole_2020_young_fb_processed.h5')

In [ ]:
tabib_2018_dir = data_dir + '/Tabib_2018'
tabib_2018_ctrl_fb = sc.read(tabib_2018_dir + '/tabib_2018_ctrl_2018_fb_processed.h5')

In [ ]:
tabib_2021_dir = data_dir + '/Tabib_2021'
tabib_2021_ctrl_fb = sc.read(tabib_2021_dir + '/tabib_2021_ctrl_fb_processed.h5')

In [ ]:
tsc_2021_dir = data_dir + '/Tabula_Sapiens_Consortium_2021'
tsc_2021_ctrl_fb = sc.read(tsc_2021_dir + '/tsc_2021_ctrl_fb_processed.h5')

In [ ]:
theo_2020_dir = data_dir + '/Theocharidis_2020/'
theo_2020_ctrl_dm_fb = sc.read(theo_2020_dir + '/theo_2020_ctrl_dm_fb_processed.h5')

In [ ]:
theo_2021_dir = data_dir + '/Theocharidis_2021/'
theo_2021_ctrl_fb = sc.read(theo_2021_dir + '/theo_2021_ctrl_fb_processed.h5')

In [ ]:
vors_2020_dir = data_dir + '/Vorstandlechner_2020'
vors_2020_ctrl_fb = sc.read(vors_2020_dir + '/vors_2020_ctrl_fb_2020_processed.h5')

## Evaluate marker score for adatas
All the **commented anndatas are discarded** for marker selection because they do not show enough marker consistency and might bias the result, or because they belong to dataset from non control/healthy conditions.

In [ ]:
list_datasets = [
#                  deng_2021_scar_fb, 
                 gao_2021_ctrl_fb, 
                 gaydosik_2020_ctrl_fb, 
#                  he_2020_ctrl_fb, 
                 hughes_2020_ctrl_fb, 
#                  kim_2020_ctrl_fb, 
                 liu_2021_ctrl_fb, 
                 mirizio_2020_scl_fb, 
#                  reynolds_2021_ctrl_fb,
                 sole_2020_young_fb, 
                 tabib_2018_ctrl_fb, 
                 tabib_2021_ctrl_fb,
#                  tsc_2021_ctrl_fb, 
                 theo_2020_ctrl_dm_fb, 
#                  theo_2021_ctrl_fb,
                 vors_2020_ctrl_fb]

list_accepted_clusters = ['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3', 'C5', 'D1', 'D2', 'D3', 'T1',]                

# manual_axis is to create a robust set of markers
for adata in list_datasets:
    adata.obs['manual_axis'] = [i[0] for i in adata.obs['cluster']]

In [ ]:
dict_make_gene_scoring = make_gene_scoring_with_expr(list_datasets=list_datasets, calculate_DEGs = True, group_name = 'cluster', value_ref = 'scores', select_method = 'pval', 
                                                     select_thres = 0.05, list_clusters=list_accepted_clusters)

In [ ]:
dict_make_gene_scoring['A1'].iloc[:40]

In [ ]:
dict_make_gene_scoring['C5']

In [ ]:
dict_make_gene_scoring['D2'].sort_values(by='Z', ascending=False)

In [ ]:
dict_make_gene_scoring_axis = make_gene_scoring_with_expr(list_datasets=list_datasets, calculate_DEGs = True, group_name = 'manual_axis', list_clusters=['A', 'B', 'C', 'D'],
                                                     value_ref = 'scores', select_method = 'pval', select_thres = 0.05)

## Recalculating clusters in the datasets

In [ ]:
dict_cats_clusters_robust = {i: np.array(dict_make_gene_scoring[i].index[:30]) for i in dict_make_gene_scoring.keys()}
dict_cats_axes_robust = {i: np.array(dict_make_gene_scoring_axis[i].index[:30]) for i in dict_make_gene_scoring_axis.keys()}

In [ ]:
dict_cats_clusters_robust

In [ ]:
dict_cats_axes_robust

### Deng 2021

In [ ]:
assign_cats(deng_2021_scar_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.8, key_added='cluster_robust', others_name='U')
assign_cats(deng_2021_scar_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

deng_2021_scar_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(deng_2021_scar_fb.obs['cluster_robust']))]
sc.pl.umap(deng_2021_scar_fb, color=['Internal sample identifier', 'leiden', 'axis_robust', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

### Gao 2021

In [ ]:
assign_cats(gao_2021_ctrl_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.9, key_added='cluster_robust', others_name='U')
assign_cats(gao_2021_ctrl_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

gao_2021_ctrl_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(gao_2021_ctrl_fb.obs['cluster_robust']))]
sc.pl.umap(gao_2021_ctrl_fb, color=['Internal sample identifier', 'leiden', 'axis_robust', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

### Gaydosik 2020

In [ ]:
assign_cats(gaydosik_2020_ctrl_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.9, key_added='cluster_robust', others_name='U')
assign_cats(gaydosik_2020_ctrl_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

gaydosik_2020_ctrl_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(gaydosik_2020_ctrl_fb.obs['cluster_robust']))]
sc.pl.umap(gaydosik_2020_ctrl_fb, color=['Internal sample identifier', 'leiden', 'axis_robust', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

### He 2020

In [ ]:
assign_cats(he_2020_ctrl_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.9, key_added='cluster_robust', others_name='U')
assign_cats(he_2020_ctrl_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

adata_he_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(he_2020_ctrl_fb.obs['cluster_robust']))]
sc.pl.umap(he_2020_ctrl_fb, color=['Internal sample identifier', 'leiden', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

### Hughes 2020

In [ ]:
assign_cats(hughes_2020_ctrl_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.9, key_added='cluster_robust', others_name='U')
assign_cats(hughes_2020_ctrl_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

adata_hughes_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(hughes_2020_ctrl_fb.obs['cluster_robust']))]
sc.pl.umap(hughes_2020_ctrl_fb, color=['Internal sample identifier', 'leiden', 'axis_robust', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

### Kim 2020

In [ ]:
assign_cats(kim_2020_ctrl_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.9, key_added='cluster_robust', others_name='U')
assign_cats(kim_2020_ctrl_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

kim_2020_ctrl_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(kim_2020_ctrl_fb.obs['cluster_robust']))]
sc.pl.umap(kim_2020_ctrl_fb, color=['Internal sample identifier', 'leiden', 'axis_robust', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

### Kim 2021

In [ ]:
assign_cats(kim_2021_ctrl_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.9, key_added='cluster_robust', others_name='U')
assign_cats(kim_2021_ctrl_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

kim_2021_ctrl_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(kim_2021_ctrl_fb.obs['cluster_robust']))]
sc.pl.umap(kim_2021_ctrl_fb, color=['Internal sample identifier', 'leiden', 'axis_robust', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

### Liu 2021

In [ ]:
assign_cats(liu_2021_ctrl_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.9, key_added='cluster_robust', others_name='U')
assign_cats(liu_2021_ctrl_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

liu_2021_ctrl_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(liu_2021_ctrl_fb.obs['cluster_robust']))]
sc.pl.umap(liu_2021_ctrl_fb, color=['Internal sample identifier', 'leiden', 'axis_robust', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

### Mirizio 2020

In [ ]:
assign_cats(mirizio_2020_scl_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.95, key_added='cluster_robust', others_name='U')
assign_cats(mirizio_2020_scl_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

mirizio_2020_scl_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(mirizio_2020_scl_fb.obs['cluster_robust']))]
sc.pl.umap(mirizio_2020_scl_fb, color=['Internal sample identifier', 'leiden', 'axis_robust', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

### Reynolds 2021

In [ ]:
assign_cats(reynolds_2021_ctrl_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.9, key_added='cluster_robust', others_name='U')
assign_cats(reynolds_2021_ctrl_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

reynolds_2021_ctrl_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(reynolds_2021_ctrl_fb.obs['cluster_robust']))]
sc.pl.umap(reynolds_2021_ctrl_fb, color=['Internal sample identifier', 'leiden', 'axis_robust', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

### Solé-Boldo 2021

In [ ]:
assign_cats(sole_2020_young_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.9, key_added='cluster_robust', others_name='U')
assign_cats(sole_2020_young_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

sole_2020_young_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(sole_2020_young_fb.obs['cluster_robust']))]
sc.pl.umap(sole_2020_young_fb, color=['Internal sample identifier', 'leiden', 'axis_robust', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

### Tabib 2018

In [ ]:
assign_cats(tabib_2018_ctrl_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.9, key_added='cluster_robust', others_name='U')
assign_cats(tabib_2018_ctrl_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

tabib_2018_ctrl_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(tabib_2018_ctrl_fb.obs['cluster_robust']))]
sc.pl.umap(tabib_2018_ctrl_fb, color=['Internal sample identifier', 'leiden', 'axis_robust', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

### Tabib 2021

In [ ]:
assign_cats(tabib_2021_ctrl_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.9, key_added='cluster_robust', others_name='U')
assign_cats(tabib_2021_ctrl_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

tabib_2021_ctrl_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(tabib_2021_ctrl_fb.obs['cluster_robust']))]
sc.pl.umap(tabib_2021_ctrl_fb, color=['Internal sample identifier', 'leiden', 'axis_robust', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

### Tabula Sapiens Consortium 2021

In [ ]:
assign_cats(tsc_2021_ctrl_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.7, key_added='cluster_robust', others_name='U')
assign_cats(tsc_2021_ctrl_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

tsc_2021_ctrl_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(tsc_2021_ctrl_fb.obs['cluster_robust']))]
sc.pl.umap(tsc_2021_ctrl_fb, color=['Internal sample identifier', 'leiden', 'axis_robust', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

### Theocarditis 2020

In [ ]:
assign_cats(theo_2020_ctrl_dm_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.9, key_added='cluster_robust', others_name='U')
assign_cats(theo_2020_ctrl_dm_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

theo_2020_ctrl_dm_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(theo_2020_ctrl_dm_fb.obs['cluster_robust']))]
sc.pl.umap(theo_2020_ctrl_dm_fb, color=['Internal sample identifier', 'leiden', 'axis_robust', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

### Theocarditis 2021

In [ ]:
assign_cats(theo_2021_ctrl_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.95, key_added='cluster_robust', others_name='U')
assign_cats(theo_2021_ctrl_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

theo_2021_ctrl_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(theo_2021_ctrl_fb.obs['cluster_robust']))]
sc.pl.umap(theo_2021_ctrl_fb, color=['Internal sample identifier', 'leiden', 'axis_robust', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

### Vorstandlechner 2020

In [ ]:
assign_cats(vors_2020_ctrl_fb, dict_cats=dict_cats_clusters_robust, min_score=0.5, quantile_gene_sel=0.9, key_added='cluster_robust', others_name='U')
assign_cats(vors_2020_ctrl_fb, column_groupby='cluster_robust', dict_cats=dict_cats_axes_robust, min_score=0.4, key_added='axis_robust', 
            intermediate_states=True, diff=0.15, others_name='U')

vors_2020_ctrl_fb.uns['cluster_robust_colors'] = [dict_colors[i] if i in dict_colors else '#bcbcbc' for i in sorted(set(vors_2020_ctrl_fb.obs['cluster_robust']))]
sc.pl.umap(vors_2020_ctrl_fb, color=['Internal sample identifier', 'leiden', 'axis_robust', 'cluster_robust'], legend_loc='on data', cmap=magma, use_raw=False, ncols=2)

## Reevaluate the presence of clusters for each dataset

In [ ]:
from fb_functions import plot_adata_cluster_properties

In [ ]:
# The structure of the dataset dict is dict: [Name, Status (healthy, young, psoriasis, etc), year, ]
list_datasets = [deng_2021_scar_fb, gao_2021_ctrl_fb, gaydosik_2020_ctrl_fb, 
                 he_2020_ctrl_fb, hughes_2020_ctrl_fb, kim_2020_ctrl_fb, 
                 liu_2021_ctrl_fb, mirizio_2020_scl_fb, reynolds_2021_ctrl_fb,
                 sole_2020_young_fb, tabib_2018_ctrl_fb, tabib_2021_ctrl_fb,
                 tsc_2021_ctrl_fb, theo_2020_ctrl_dm_fb, theo_2021_ctrl_fb,
                 vors_2020_ctrl_fb]

list_names = [adata.obs['Author'].values[0] for adata in list_datasets]

In [ ]:
plot_adata_cluster_properties(dict_cats_clusters=dict_cats_clusters_robust, list_datasets=list_datasets, what='presence', cluster_name='cluster', axis_name='axis')

In [ ]:
plot_adata_cluster_properties(dict_cats_clusters=dict_cats_clusters_robust, list_datasets=list_datasets, what='percentage', cluster_name='cluster', axis_name='axis')

In [ ]:
plot_adata_cluster_properties(dict_cats_clusters=dict_cats_clusters_robust, list_datasets=list_datasets, what='axis', cluster_name='cluster', axis_name='axis')

## Plotting all Adatas

In [ ]:
fig, axs = plt.subplots(4, 5, figsize=(5 * 4, 4 * 4))

for adata, name, idx in zip(list_datasets, list_names, range(len(list_datasets))):
    sc.pl.umap(adata, color=['cluster'], legend_loc='on data', show=False, ax = axs.ravel()[idx], title=name, size=15, cmap=magma)

In [ ]:
fig, axs = plt.subplots(4, 5, figsize=(5 * 4, 4 * 4))

for adata, name, idx in zip(list_datasets, list_names, range(len(list_datasets))):
    sc.tl.dendrogram(adata, groupby='cluster')
    sc.pl.dendrogram(adata, groupby='cluster', show=False, ax = axs.ravel()[idx])

## PAGA 

In [ ]:
fig, axs = plt.subplots(4, 5, figsize=(5 * 4, 4 * 4))

for adata, name, idx in zip(list_datasets, list_names, range(len(list_datasets))):
    sc.tl.paga(adata, groups='cluster')
    sc.pl.paga(adata, ax=axs.ravel()[idx], frameon=False, show=False, title=name)

In [ ]:
fig, axs = plt.subplots(4, 5, figsize=(5 * 4, 4 * 4))

for adata, name, idx in zip(list_datasets, list_names, range(len(list_datasets))):
    sc.tl.paga(adata, groups='cluster')
    sc.pl.paga(adata, ax=axs.ravel()[idx], frameon=False, show=False, solid_edges='connectivities_tree', title=name)